In [12]:
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import gzip, pickle
from sklearn.preprocessing import OneHotEncoder

In [ ]:
%matplotlib inline

In [23]:
def load_data():
    with gzip.open('/home/aaditya/.keras/datasets/mnist.pkl.gz', 'rb') as f:
        train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
    x_train, y_train = train_set
    x_val, y_val = valid_set
    x_test, y_test = test_set

    x_train = np.array(x_train, dtype='float32').reshape(-1,28,28)
    x_val = np.array(x_val, dtype='float32').reshape(-1,28,28)
    x_test = np.array(x_test, dtype='float32').reshape(-1,28,28)

    y_train = OneHotEncoder(dtype=np.float32, sparse=False).fit_transform(y_train.reshape(-1,1))
    y_val = OneHotEncoder(dtype=np.float32, sparse=False).fit_transform(y_val.reshape(-1,1))
    y_test = OneHotEncoder(dtype=np.float32, sparse=False).fit_transform(y_test.reshape(-1,1))
    
    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

# def convert(size, box):
#     dw = 1./size[0]
#     dh = 1./size[1]
#     x = (box[0] + box[1])/2.0
#     y = (box[2] + box[3])/2.0
#     w = box[1] - box[0]
#     h = box[3] - box[2]
#     x = x*dw
#     w = w*dw
#     y = y*dh
#     h = h*dh
#     return (x,y,w,h)

In [33]:
def create_img(data_array, label_array):
    screen = (86,86)
    img = Image.new('F', screen)
    mnist_img = Image.fromarray(data_array*255)
    # mnist_img = mnist_img.crop((2,2,26,26))

    w = int(round(mnist_img.width * np.random.uniform(1.0, 3.0)))
    mnist_img = mnist_img.resize((w,w), Image.BICUBIC)

    x = np.random.randint(0, img.width-w)
    y = np.random.randint(0, img.height-w)
    img.paste(mnist_img, (x, y))
    
    dw = 1./screen[0]
    x = x + w/2; y = y + w/2
    x = x*dw; y = y*dw
    return np.array(img, dtype='float32').clip(0,255).reshape(86,86,1) /255 , np.concatenate([label_array, np.array([x, y, w*dw], dtype='float32')])

In [31]:
class DataGenerator():
    def __init__(self, data_array, label_array, batch_size=32):
        self.array = data_array
        self.label_array = label_array
        self.batch_size = batch_size
        
    def gen_batch(self):
        img_batch = []; coord_batch = []
        idx = [np.random.randint(0, len(self.array)-self.batch_size) for i in range(self.batch_size)]
        for i in idx:
            img, coord = create_img(self.array[i], self.label_array[i])
            img_batch.append(img); coord_batch.append(coord)
        return np.stack(img_batch), np.stack(coord_batch)
    
    def flow(self):
        while True:
            yield self.gen_batch()

a = DataGenerator(x_train, y_train).gen_batch()

a[0].shape